# Data processing

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import pandas as pd
import numpy as np

In [178]:
df = pd.read_csv('/content/drive/MyDrive/2024 datathon/Train/chevron_training.csv')


In [179]:

# Set the threshold for missing proportion. reference:
# https://www.researchgate.net/figure/Effect-of-threshold-selection-on-the-relative-proportion-of-missing-data-and-the-number_fig1_327942436
missing_threshold = 0.3


columns_to_drop = df.columns[df.isnull().mean() >= missing_threshold]
df_dropped = df.drop(columns=columns_to_drop)
columns_to_drop


Index(['number_of_stages', 'total_proppant', 'total_fluid', 'ffs_frac_type',
       'proppant_intensity', 'frac_fluid_intensity', 'average_stage_length',
       'average_proppant_per_stage', 'average_frac_fluid_per_stage',
       'proppant_to_frac_fluid_ratio', 'frac_fluid_to_proppant_ratio',
       'relative_well_position', 'batch_frac_classification',
       'well_family_relationship', 'frac_seasoning', 'OilPeakRate'],
      dtype='object')

In [180]:
# Impute missing values for numerical columns with median when the data distribution is skewed or contains outliers.
numerical_columns = df.select_dtypes(include='float64').columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())



In [181]:
# Create mappings (prepare for neural network imputation)
ffs_frac_type_mapping = {
    'Slickwater Only': 1,
    'Slickwater Crosslink Hybrid': 2,
    'Slickwater Linear Hybrid': 3,
    'Undefined': 4,
    'Crosslink Only': 5,
    'Linear Only': 6
}


df['ffs_frac_type'] = df['ffs_frac_type'].map(ffs_frac_type_mapping)


relative_well_position_mapping = {
    'Standalone Well': 1,
    'Outer Well': 2,
    'Inner Well': 3,
    'Unknown': 4
}

df['relative_well_position'] = df['relative_well_position'].map(relative_well_position_mapping)


batch_frac_classification_mapping = {
    'Non-Batch Frac': 1,
    'Unknown': 2,
    'Batch-Concurrent Frac': 3,
    'Batch-Sequential Frac': 4
}
df['batch_frac_classification'] = df['batch_frac_classification'].map(batch_frac_classification_mapping)


well_family_relationship_mapping = {
    'Standalone Well': 1,
    'Sibling Well': 2,
    'Infill Child Well': 3,
    'Unknown': 4
}

df['well_family_relationship'] = df['well_family_relationship'].map(well_family_relationship_mapping)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

categorical_columns_with_missing = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship']

target_column_mappings = {
    'ffs_frac_type': {'Slickwater Only': 1, 'Slickwater Crosslink Hybrid': 2, 'Slickwater Linear Hybrid': 3, 'Undefined': 4, 'Crosslink Only': 5, 'Linear Only': 6},
    'relative_well_position': {'Standalone Well': 1, 'Outer Well': 2, 'Inner Well': 3, 'Unknown': 4},
    'batch_frac_classification': {'Non-Batch Frac': 1, 'Unknown': 2, 'Batch-Concurrent Frac': 3, 'Batch-Sequential Frac': 4},
    'well_family_relationship': {'Standalone Well': 1, 'Sibling Well': 2, 'Infill Child Well': 3, 'Unknown': 4}
}

df.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in categorical_columns_with_missing:
    if column in df.columns:
        X = df.drop([column], axis=1)
        y = df[column]

        if y.isnull().any():
            not_nan_indices = ~y.isnull()
            X = X[not_nan_indices]
            y = y[not_nan_indices]

        known_values = df.dropna(subset=[column])
        unknown_values = df[df[column].isnull()]
        imputer = SimpleImputer(strategy='mean')
        X_imputed = imputer.fit_transform(X)
        unknown_values_imputed = imputer.transform(unknown_values.drop([column], axis=1))

        regression_model = LinearRegression()
        regression_model.fit(X_imputed, y)

        # Predict missing values
        predicted_values = regression_model.predict(unknown_values_imputed)
        df.loc[df[column].isnull(), column] = predicted_values

        # Reverse mapping for the column
        reverse_mapping_column = {v: k for k, v in target_column_mappings[column].items()}

        df[column] = df[column].map(reverse_mapping_column)



In [182]:
categorical_columns_with_missing = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship']

random_indices = np.random.choice(df.index, size=df[categorical_columns_with_missing].isnull().sum().sum(), replace=True)


for column in categorical_columns_with_missing:
    unique_categories = df[column].dropna().unique()
    missing_mask = df[column].isnull()
    df.loc[missing_mask, column] = np.random.choice(unique_categories, size=missing_mask.sum(), replace=True)



In [183]:
key_numerical_variables = ['total_fluid', 'gross_perforated_length',  'true_vertical_depth']
categorical_variables = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship', ]

# bin_lateral_length' and 'frac_fluid_intensity' are too correlated with others

In [184]:
relevant_columns = key_numerical_variables + categorical_variables + ['OilPeakRate'] + ['bin_lateral_length', 'frac_fluid_intensity'] # TODO
df = df[relevant_columns]
relevant_columns #final dataset columns

['total_fluid',
 'gross_perforated_length',
 'true_vertical_depth',
 'ffs_frac_type',
 'relative_well_position',
 'batch_frac_classification',
 'well_family_relationship',
 'OilPeakRate',
 'bin_lateral_length',
 'frac_fluid_intensity']

In [185]:


categorical_columns = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship', ]
df_dummies_list = [pd.get_dummies(df[column], prefix=column, drop_first=True) for column in categorical_columns]

# Concatenate dummy variables back to the original DataFrame
df_combined = pd.concat([df] + df_dummies_list, axis=1)
df = df_combined

In [186]:
import pandas as pd



num_bins = 3
bins = pd.cut(df['gross_perforated_length'], bins=num_bins, labels=False)


df['gross_perforated_length_bins'] = bins




In [187]:
# find difference between df.columns and df_test.columns
diff = set(df.columns) - set(df_test.columns)
diff

{'OilPeakRate', 'ffs_frac_type_6.0'}

In [188]:
# find difference between df.columns and df_test.columns
diff = set(df_test.columns) - set(df.columns)
diff

set()

In [190]:
# drop 'ffs_frac_type_Linear Only'
df = df.drop('ffs_frac_type_6.0', axis=1)

In [191]:
df.shape

(29692, 24)

In [150]:
df_test.shape

(400, 23)

In [146]:
df.columns

Index(['total_fluid', 'gross_perforated_length', 'true_vertical_depth',
       'ffs_frac_type', 'relative_well_position', 'batch_frac_classification',
       'well_family_relationship', 'OilPeakRate', 'bin_lateral_length',
       'frac_fluid_intensity', 'ffs_frac_type_2.0', 'ffs_frac_type_3.0',
       'ffs_frac_type_4.0', 'ffs_frac_type_5.0', 'relative_well_position_2.0',
       'relative_well_position_3.0', 'relative_well_position_4.0',
       'batch_frac_classification_2.0', 'batch_frac_classification_3.0',
       'batch_frac_classification_4.0', 'well_family_relationship_2.0',
       'well_family_relationship_3.0', 'well_family_relationship_4.0',
       'gross_perforated_length_bins'],
      dtype='object')

In [149]:
df_test.columns

Index(['total_fluid', 'gross_perforated_length', 'true_vertical_depth',
       'ffs_frac_type', 'relative_well_position', 'batch_frac_classification',
       'well_family_relationship', 'bin_lateral_length',
       'frac_fluid_intensity', 'ffs_frac_type_2', 'ffs_frac_type_3',
       'ffs_frac_type_4', 'ffs_frac_type_5', 'relative_well_position_2',
       'relative_well_position_3', 'relative_well_position_4',
       'batch_frac_classification_2', 'batch_frac_classification_3',
       'batch_frac_classification_4', 'well_family_relationship_2',
       'well_family_relationship_3', 'well_family_relationship_4',
       'gross_perforated_length_bins'],
      dtype='object')

# model

In [100]:
# import pandas as pd
# df2 = pd.read_csv('/content/cleaned.csv')
# df2.columns

In [192]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np




X = df.drop('OilPeakRate', axis=1)
y = df['OilPeakRate']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


rf_model = RandomForestRegressor(n_estimators=10, random_state=42)
xgb_model = XGBRegressor(random_state=42)
nn_model = LinearRegression()

# Stacking
stacking_model = StackingRegressor(
    estimators=[ ('nn', nn_model),('rf', rf_model), ('xgb', xgb_model)],
    final_estimator=LinearRegression()
)
stacking_model.fit(X_train, y_train)
stacking_predictions2 = stacking_model.predict(X_test)
stacking_rmse = np.sqrt(mean_squared_error(y_test, stacking_predictions2))
print(f"Stacking (Random Forest + XGBoost + LinearRegression) RMSE: {stacking_rmse}")



Stacking (Random Forest + XGBoost + LinearRegression) RMSE: 96.80822462817179


# test

In [163]:
df_test = pd.read_csv('/content/scoring.csv')

relevant_columns = ['total_fluid',
 'gross_perforated_length',
 'true_vertical_depth',
 'ffs_frac_type',
 'relative_well_position',
 'batch_frac_classification',
 'well_family_relationship',
 'bin_lateral_length',
 'frac_fluid_intensity']


# Create mappings (prepare for neural network imputation)
ffs_frac_type_mapping = {
    'Slickwater Only': 1.0,
    'Slickwater Crosslink Hybrid': 2.0,
    'Slickwater Linear Hybrid': 3.0,
    'Undefined': 4.0,
    'Crosslink Only': 5.0,
    'Linear Only': 6.0
}


df_test['ffs_frac_type'] = df_test['ffs_frac_type'].map(ffs_frac_type_mapping)


relative_well_position_mapping = {
    'Standalone Well': 1.0,
    'Outer Well': 2.0,
    'Inner Well': 3.0,
    'Unknown': 4.0
}

df_test['relative_well_position'] = df_test['relative_well_position'].map(relative_well_position_mapping)


batch_frac_classification_mapping = {
    'Non-Batch Frac': 1.0,
    'Unknown': 2.0,
    'Batch-Concurrent Frac': 3.0,
    'Batch-Sequential Frac': 4.0
}
df_test['batch_frac_classification'] = df_test['batch_frac_classification'].map(batch_frac_classification_mapping)


well_family_relationship_mapping = {
    'Standalone Well': 1.0,
    'Sibling Well': 2.0,
    'Infill Child Well': 3.0,
    'Unknown': 4.0
}

df_test['well_family_relationship'] = df_test['well_family_relationship'].map(well_family_relationship_mapping)


categorical_columns_with_missing = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship']

random_indices = np.random.choice(df.index, size=df[categorical_columns_with_missing].isnull().sum().sum(), replace=True)


for column in categorical_columns_with_missing:
    unique_categories = df_test[column].dropna().unique()
    missing_mask = df_test[column].isnull()
    df_test.loc[missing_mask, column] = np.random.choice(unique_categories, size=missing_mask.sum(), replace=True)


key_numerical_variables = ['total_fluid', 'gross_perforated_length',  'true_vertical_depth']
categorical_variables = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship', ]

relevant_columns = key_numerical_variables + categorical_variables + ['bin_lateral_length', 'frac_fluid_intensity'] # TODO
df_test = df_test[relevant_columns]
relevant_columns


categorical_columns = ['ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship', ]
df_test_dummies_list = [pd.get_dummies(df_test[column], prefix=column, drop_first=True) for column in categorical_columns]






In [164]:

# Concatenate dummy variables back to the original DataFrame
df_test_combined = pd.concat([df_test] + df_test_dummies_list, axis=1)
df_test = df_test_combined


num_bins = 3
bins = pd.cut(df_test['gross_perforated_length'], bins=num_bins, labels=False)

df_test['gross_perforated_length_bins'] = bins

# drop = ['gross_perforated_length','ffs_frac_type', 'relative_well_position', 'batch_frac_classification', 'well_family_relationship', ]
# df_test = df_test.drop(drop, axis=1)


In [193]:
df

,total_fluid,gross_perforated_length,true_vertical_depth,ffs_frac_type,relative_well_position,batch_frac_classification,well_family_relationship,OilPeakRate,bin_lateral_length,frac_fluid_intensity,...,relative_well_position_2.0,relative_well_position_3.0,relative_well_position_4.0,batch_frac_classification_2.0,batch_frac_classification_3.0,batch_frac_classification_4.0,well_family_relationship_2.0,well_family_relationship_3.0,well_family_relationship_4.0,gross_perforated_length_bins
0,3.564951e+06,3963.582677,8712.598425,3.0,1.0,2.0,1.0,46.623023,1.0,585.580726,...,0,0,0,1,0,0,0,0,0,0
1,5.610964e+05,3179.133858,8627.952756,3.0,2.0,2.0,3.0,59.750009,1.0,176.493480,...,1,0,0,1,0,0,0,1,0,0
2,1.621424e+06,3810.039370,8801.181102,1.0,1.0,1.0,1.0,10.785716,1.0,425.566078,...,0,0,0,0,0,0,0,0,0,0
3,4.377134e+05,3723.425197,8635.826772,5.0,1.0,2.0,1.0,123.797638,1.0,117.556659,...,0,0,0,1,0,0,0,0,0,0
4,6.229802e+05,3811.023622,8664.370079,4.0,2.0,2.0,3.0,102.309540,1.0,163.467933,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29687,3.564951e+06,6684.055118,9148.622047,6.0,1.0,4.0,1.0,192.767887,1.5,585.580726,...,0,0,0,0,0,1,0,0,0,1
29688,3.564951e+06,6684.055118,9148.622047,5.0,2.0,3.0,4.0,192.767887,1.5,585.580726,...,1,0,0,0,1,0,0,0,1,1
29689,3.564951e+06,6684.055118,9148.622047,1.0,1.0,4.0,3.0,192.767887,1.5,585.580726,...,0,0,0,0,0,1,0,1,0,1
29690,3.564951e+06,6684.055118,9148.622047,2.0,1.0,3.0,1.0,192.767887,1.5,585.580726,...,0,0,0,0,1,0,0,0,0,1


In [165]:
df_test

,total_fluid,gross_perforated_length,true_vertical_depth,ffs_frac_type,relative_well_position,batch_frac_classification,well_family_relationship,bin_lateral_length,frac_fluid_intensity,ffs_frac_type_2.0,...,relative_well_position_2.0,relative_well_position_3.0,relative_well_position_4.0,batch_frac_classification_2.0,batch_frac_classification_3.0,batch_frac_classification_4.0,well_family_relationship_2.0,well_family_relationship_3.0,well_family_relationship_4.0,gross_perforated_length_bins
0,2.049015e+06,6370.078740,8337.598425,2.0,2.0,1.0,2.0,1.5,321.662440,1,...,1,0,0,0,0,0,1,0,0,1
1,1.876124e+06,5244.094488,8808.070866,2.0,1.0,1.0,1.0,1.0,357.759307,1,...,0,0,0,0,0,0,0,0,0,1
2,2.773870e+06,6694.881890,8638.779528,2.0,2.0,1.0,3.0,1.5,414.326910,1,...,1,0,0,0,0,0,0,1,0,1
3,2.773870e+06,6694.881890,8536.417323,2.0,1.0,2.0,1.0,1.5,414.326910,1,...,0,0,0,1,0,0,0,0,0,1
4,1.340587e+06,2520.669291,8668.307087,2.0,1.0,2.0,1.0,1.0,531.837898,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1.163932e+06,4302.165354,5226.377953,5.0,2.0,2.0,3.0,1.0,270.545556,0,...,1,0,0,1,0,0,0,1,0,0
396,1.303240e+06,4342.519685,5245.078740,5.0,1.0,1.0,1.0,1.0,300.111524,0,...,0,0,0,0,0,0,0,0,0,0
397,2.174185e+06,7380.905512,5288.385827,5.0,1.0,2.0,1.0,1.5,294.568916,0,...,0,0,0,1,0,0,0,0,0,2
398,1.035400e+06,7357.283465,5109.251969,5.0,1.0,2.0,1.0,1.5,140.731313,0,...,0,0,0,1,0,0,0,0,0,2


In [197]:
# standard scale df_test
df_test = scaler.transform(df_test)
stacking_predictions = stacking_model.predict(df_test)
stacking_predictions


array([145.42128254, 135.42724066, 162.97099102, 143.23432279,
        65.36974971, 115.66196984, 113.56775066, 150.97605212,
       183.4105496 , 123.65128648, 137.3828547 , 121.92104181,
        67.44082453, 131.11125702, 197.72752492, 169.21166032,
        29.57935537,  79.68189913, 110.6516797 , 245.86381226,
        86.58255883, 212.38716304, 145.91585077, 249.39866947,
       282.75309368, 127.37464745, 129.82495766, 235.11908703,
       228.30813469, 218.18816899, 212.61077291, 178.23003899,
       143.04572855, 134.63791987,  79.20637341, 277.15762599,
       138.22940986, 164.38238748, 162.47268345, 251.68928616,
       244.63559887, 265.97116204, 303.90118892, 353.68229491,
       256.76676921, 227.35262272, 440.43228922, 346.13054614,
       263.64999905, 222.1626722 , 232.45582031, 219.17689158,
       491.65907286, 227.76281929, 294.25676052, 322.40424132,
       136.94323247, 333.95779198, 449.22244673, 167.36147346,
       268.01480519, 155.66205102, 339.04707295, 267.31

In [200]:
stacking_predictions_df

,0
0,145.421283
1,135.427241
2,162.970991
3,143.234323
4,65.369750
...,...
395,136.548036
396,103.264978
397,105.390347
398,126.521824
